In [17]:
from typing import TypeAlias, Union, Sequence, Any, Mapping, Optional
import psycopg
from psycopg import sql
import io
import boto3

Query: TypeAlias = Union[bytes, "sql.SQL", "sql.Composed"]
Params: TypeAlias = Union[Sequence[Any], Mapping[str, Any]]

In [2]:
pg_connection_dict = {
    'dbname': "mydb",
    'user': "myn",
    'password': r"2)2K9zJCKZv7pLUd",
    'port': "5432",
    'host': "terraform-20221222010822007100000002.c2x7llrlmsr3.us-east-2.rds.amazonaws.com"
}

In [18]:
bucket_name = "sto01.dev.us-east-2.aws.shouldishovel.com"
s3 = boto3.client('s3')
s3.download_file(bucket_name, "temp.csv", 'temp.csv')

In [19]:
with open("./temp.csv", "rt") as f:
    csv_buffer = io.StringIO(f.read())

In [20]:
def execute_copy_statement_using_file(
    conn_details: dict[str, str],
    sql_statement: Query,
    csv_file: io.StringIO,
    params: Optional[Params] = None,
) -> None:
    """Execute sql copy statement against the conn_details using the DataFrame provided"""
    with psycopg.connect(**conn_details, autocommit=True) as conn:
        with conn.cursor() as curr:
            with curr.copy(sql_statement, params) as copy:
                csv_file.seek(0)
                copy.write(csv_file.read())
            print(f"Rows impacted: {curr.rowcount}")

copy_statement = sql.SQL("""
    COPY {table} FROM stdin (format csv, delimiter ' ', header true)
""").format(
    table=sql.Identifier("temp")
)

In [21]:
execute_copy_statement_using_file(pg_connection_dict, copy_statement, csv_buffer)

Rows impacted: 3377217
